# Iniciar Modelo IA - Gemini 🤖

Configura e Inicia a IA

In [30]:
from pathlib import Path
import hashlib
import google.generativeai as genai
import textwrap
from IPython.display import display, Markdown

genai.configure(api_key="SUA_API_KEY")

# Set up the model
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 0,
  "max_output_tokens": 8192,
}

safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
]

uploaded_files = []
def upload_if_needed(pathname: str) -> list[str]:
  path = Path(pathname)
  hash_id = hashlib.sha256(path.read_bytes()).hexdigest()
  try:
    existing_file = genai.get_file(name=hash_id)
    return [existing_file]
  except:
    pass
  uploaded_files.append(genai.upload_file(path=path, display_name=hash_id))
  return [uploaded_files[-1]]


# Importando os arquivos de dados para análise, tive que fazer assim porque não foi possível fazer o upload dos .csv

clientes = []
nota_produtos = []
produtos = []
notas = []

with open ('CLIENTES.CSV', encoding='utf-8') as cliente_csv:
    for lst in cliente_csv:
      clientes.append(lst)
      
with open ('NOTAS.CSV', encoding='utf-8') as notas_csv:
    for lst in notas_csv:
      notas.append(lst)
      
with open ('PRODUTOS.CSV', encoding='ansi') as produtos_csv:
    for lst in produtos_csv:
      produtos.append(lst)  
      
with open ('NOTA_PRODUTOS.CSV', encoding='utf-8') as nota_produtos_csv:
    for lst in nota_produtos_csv:
      nota_produtos.append(lst)                       

lista_clientes = ''
lista_notas =''
lista_produtos =''
lista_nota_produtos =''

for lst in clientes:
  lista_clientes += lst

for lst in notas:
  lista_notas += lst
  
for lst in produtos:
  lista_produtos += lst
  
for lst in nota_produtos:
  lista_nota_produtos += lst    

# Criação do historico para a IA saber como gerar os relatórios

historico = [
  {
    "role": "user",
    "parts": "lista de clientes em CSV"},
  {
    "role": "user",
    "parts": {lista_clientes}},  
    {"role": "user",
    "parts": "baseado na lista de clientes quais os clientes de Santa Catarina"}, 
    {"role": "model",
    "parts": """
  # Clientes de Santa Catarina  
    
| CODIGO | NOME | ENDERECO | CIDADE | ESTADO |
| --- | --- | --- | --- | --- |
| 75 | Ana Costa | Rua da Praia, 33 | Florianópolis | Santa Catarina |
| 117 | Vinicius Oliveira | Travessa da Paz, 365 | Florianópolis	  Santa Catarina |
| 170 | Bernardo Oliveira | Travessa da Paz, 871 | Florianópolis	  Santa Catarina |
| 185 | Benício Gomes | Rua da Praia, 1069 | Florianópolis	  Santa Catarina |
| 5925 | Caleb Oliveir | Travessa da Paz, 1553 | Florianópolis	  Santa Catarina |"""
  },
  {
    "role": "user",
    "parts": "lista de notas em CSV"},    
  {
    "role": "user",
    "parts": {lista_notas}},
  {
    "role": "user",
    "parts": "o codigo do cliente na nota tem o mesmo codigo na lista de clientes"},  
    {"role": "user",
    "parts": "baseado na lista de notas Vendas em Dezembro de 2023"}, 
    {"role": "model",
    "parts": """
# Vendas em Dezembro de 2023 
     
|NUMERO | EMISSAO | CLIENTE | VALOR_TOTAL | CANCELADA |
| --- | --- | --- | --- | --- |
|2886 | 18/12/2023 | Lara Correia | 180.420 | N |
|2877 | 07/12/2023 | Lara Correia | 227.010 | N |
|2884 | 18/12/2023 | Samuel Rocha | 2.145.490 | S |
| | | | |TOTAL: 407,43| |"""
  },
  {
    "role": "user",
    "parts": "Vendas para cliente Marina Santos"},
    {"role": "model",
    "parts": """
# Vendas para cliente Marina Santos  
    
| NUMERO | EMISSAO | CLIENTE | VALOR_TOTAL |
| --- | --- | --- | --- |
| 583 | 22/09/2015 | Marina Santos | 414.180 |	
| 713 | 11/04/2016 | Marina Santos | 592.600 |	
| 799 | 05/08/2016 | Marina Santos | 615.410 |
| 841 | 12/09/2016 | Marina Santos | 689.420 |	
| | | | Total: 2311,61 |    
"""},
  {
    "role": "user",
    "parts": "Vendas por cliente em maio de 2023"},
    {"role": "model",
    "parts": """
# Vendas por Cliente Maio de 2023   
    
| NUMERO | EMISSAO | CLIENTE | NOME | VALOR_TOTAL | CANCELADA |
| --- | --- | --- | --- | --- | --- |
| 2736 | 22/05/2023 | 128 | Samuel Rocha | 2.785.000 | S | 
| | | | | Total Samuel: 0.00 | |
||
| 2721 | 09/05/2023 | 216 | Lara Correia | 76.850 | N | 
| 2722 | 09/05/2023 | 216 | Lara Correia | 55.880 | N |
| 2723 | 09/05/2023 | 216 | Lara Correia | 563.780 | N |
| | | | | Total Lara: 696.510 | |
||
| 2733 | 17/05/2023 | 5925 | Caleb Oliveira | 201.590 | N |
| 2731 | 17/05/2023 | 5925 | Caleb Oliveira | 531.690 | N |
| 2732 | 17/05/2023 | 5925 | Caleb Oliveira | 195.430 | N |
# | | | | | Total Lara: 928.710 | |"""},
  {
    "role": "user",
    "parts": lista_produtos},
  {
    "role": "user",
    "parts": lista_notas},
  {
    "role": "user",
    "parts": "baseado na lista de notas Vendas dos produtos de Higiene em Fevereiro 2022"},
  {"role": "model",
  "parts":"""
# Vendas de produtos de Higiene em Fevereiro 2022 
|PRODUTO|DESCRICAO|QUANTIDADE|TOTAL|
|---|---|---|---|
|915307|PAPEL HIG SUBLIME NEUTRO PG11 LV12|1|10.65|
| | | | TOTAL: 10.65|"""}              
]

def adiciona_prompt_parts(prompt_parts,pergunta): 
  prompt_parts.append(f'input : {pergunta}')
  prompt_parts.append("output: ")  
  
  return prompt_parts

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

# Relatório 📄

## Solicitar Relatório

Digitar qual relatório deseja gerar

In [31]:
model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest",
                              generation_config=generation_config,
                              safety_settings=safety_settings)

chat = model.start_chat(history=historico)

pergunta = input('Qual relatório precisa gerar? ')

resposta = chat.send_message(pergunta)

display(to_markdown(resposta.text))

> # Vendas de Dezembro 2022 
> 
> |NUMERO | EMISSAO | CLIENTE | NOME | VALOR_TOTAL | CANCELADA |
> | --- | --- | --- | --- | --- | --- |
> | 2618 | 19/12/2022 | 216 | Lara Correia | 448.700 | N |
> | 2617 | 19/12/2022 | 216 | Lara Correia | 61.300 | N |
> | 2616 | 19/12/2022 | 216 | Lara Correia | 266.830 | N |
> | 2615 | 19/12/2022 | 216 | Lara Correia | 557.180 | N |
> | 2614 | 19/12/2022 | 216 | Lara Correia | 242.630 | N |
> |  |  |  | Total Lara: 1.576.640 |  |
> || 
> | 2607 | 06/12/2022 | 5925 | Caleb Oliveira | 70.310 | N |
> | 2606 | 06/12/2022 | 5925 | Caleb Oliveira | 69.560 | N |
> | 2605 | 06/12/2022 | 5925 | Caleb Oliveira | 355.580 | N |
> |  |  |  | Total Caleb:  505.450 |  |
> || 
> |  |  |  | TOTAL GERAL: 2.082.090 |  |
> 


## Análise do Relatório

Chat para analisar o resultado

In [34]:
pergunta = input('Fazer perguntas sobre o relatório? (Digite ''nao'' para finalizar)').lower()

while pergunta != 'nao':
    resposta = chat.send_message(pergunta)
    display(to_markdown(resposta.text))
    pergunta = input('Fazer perguntas sobre o relatório? (Digite ''nao'' para finalizar)').lower()

> ## Análise Curta das Vendas - Dezembro 2022
> 
> **Dezembro de 2022 teve um desempenho positivo, com destaque para a cliente Lara Correia, responsável por 75,7% do faturamento.** 
> 
> No entanto, é crucial **diversificar a base de clientes** para evitar a dependência excessiva.  **Investigar os cancelamentos de Caleb Oliveira (24,3% das vendas)**  é fundamental para evitar perdas futuras. 
